In [1]:
from collections import Counter

import torch
import torch.nn as nn
import torch.nn.functional as F
import numpy as np

In [ ]:
import numpy as np
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense, Embedding
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
import pandas as pd

filtered_quotes_and_authors = pd.read_csv('C:/Users/Hp/Downloads/filtered_quotes_and_authors.csv')

device = torch.device('cuda') if torch.cuda.is_available() else torch.device('cpu')
# Предобработка текста
quotes = filtered_quotes_and_authors['quote'].tolist()
tokenizer = Tokenizer()
tokenizer.fit_on_texts(quotes)
sequences = tokenizer.texts_to_sequences(quotes)
vocab_size = len(tokenizer.word_index) + 1
max_sequence_len = max([len(seq) for seq in sequences])

# Создание обучающих последовательностей
input_sequences = []
output_sequences = []
for seq in sequences:
    for i in range(1, len(seq)):
        input_sequences.append(seq[:i])
        output_sequences.append(seq[i])

# Подготовка входных и выходных данных
X = pad_sequences(input_sequences, maxlen=max_sequence_len)
y = np.array(output_sequences)
y = np.expand_dims(y, axis=-1)

# Создание модели LSTM
model = Sequential()
model.add(Embedding(vocab_size, 100, input_length=max_sequence_len))
model.add(LSTM(150))
model.add(Dense(vocab_size, activation='softmax'))
model.compile(loss='sparse_categorical_crossentropy', optimizer='adam')


# Обучение модели
model.fit(X, y, epochs=4, verbose=1)

# Генерация текста
seed_text = "Darling <NAME>,"
next_words = 20

for _ in range(next_words):
    token_list = tokenizer.texts_to_sequences([seed_text])[0]
    token_list = pad_sequences([token_list], maxlen=max_sequence_len-1)
    predicted = model.predict_classes(token_list, verbose=0)
    
    output_word = ""
    for word, index in tokenizer.word_index.items():
        if index == predicted:
            output_word = word
            break
    seed_text += " " + output_word

print(seed_text)


Epoch 1/4
 114/2656 [>.............................] - ETA: 6:26:14 - loss: 6.9649